# Decision trees
Author: Fadoua Ghourabi (fadouaghourabi@gmail.com)

Date: June 20, 2019

**Warm-up practice.** 
1. I pick one animal among _dolphin_, _bear_, _owl_, _penguin_. Your goal is the guess which animal I picked by asking yes/no questions. 
2. Familiarize yourself with common terminology of binary trees, e.g. **root**, **node**, **leaf**, **depth**, **child node**, **parent node**, etc. 

A decision tree is a binary tree that splits the dataset into subset by performing a simpe if/else test. It is used for both classification and regression. We first explain the classification with a decision tree.

### An example to start

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer

In [ ]:
cancer = load_breast_cancer()
print(cancer.DESCR)

In [ ]:
cancer

In [ ]:
X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = pd.DataFrame(cancer.target, columns=["Type"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=412)

In [ ]:
tree = DecisionTreeClassifier(random_state=12)
tree.fit(X_train, y_train)

In [ ]:
train_score = tree.score(X_train,y_train)
test_score = tree.score(X_test,y_test)
print("Score of train set: {}".format(train_score))
print("Score of test set: {}".format(test_score))

### Pruning

The accuracy of the training set is 100%. The tree was grown deep enough that each leaf is **pure**, in other words, it contains one class. This situation is overfitting (test score << train score). We restrict the depth of a decision tree to improve the model, which is called **pruning**.

In [ ]:
tree = DecisionTreeClassifier(max_depth=4, random_state=12)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

In [ ]:
train_score = tree.score(X_train,y_train)
test_score = tree.score(X_test,y_test)
print("Score of train set: {}".format(train_score))
print("Score of test set: {}".format(test_score))

We can visualize the decision tree using ``export_graphviz``. 

In [ ]:
from sklearn.tree import export_graphviz
export_graphviz(tree, out_file="tree.dot", class_names=["malignant", "benign"],
                feature_names=cancer.feature_names, impurity=False, filled=True)

In [ ]:
import graphviz

with open("tree.dot") as f:
    dot_graph = f.read()
display(graphviz.Source(dot_graph))

Each node includes the following information:
- The splitting feature, e.g. root splits the data to (``worst concave points <= 0.147``) and (``worst concave points > 0.147``)
- ``samples`` is the count of input samples
- ``value`` is the output, or the count of samples in each class. For instance, ``value = [23, 263]`` means 23 samples belong to class malignant and 263 samples to class benign.
- ``class`` indicates the dominant class. 

**Practice.** Change the depth. What do you observe?

### Impurity metrics

#### Gini index

The **gini index** is a metric that quantifies the purity of a node or a leaf. A gini score greater than zero implies that samples contained within that node belong to different classes. A gini score of zero means that the node is pure, that within that node only a single class of samples exist. 

To display the gini score, in function ``export_graphviz`` change the ``impurity`` to True.

In [ ]:
export_graphviz(tree, out_file="tree.dot", class_names=["malignant", "benign"],
                feature_names=cancer.feature_names, impurity=True, filled=True)
with open("tree.dot") as f:
    dot_graph = f.read()
display(graphviz.Source(dot_graph))

The gini index at node $n$ is given by $\sum_{k\neq k'} p_{nk} * p_{nk'}$, where $k$ and $k'$ are classes and $p_{nk}$ is the probability of class $k$.

**Practice.** Verify the gini indexes of the tree above.

#### Entropy 

Intuitively, entropy measures disorder or how messy your data is. In decision trees, the goal is to tidy the data. You try to separate your data and group the samples together in the classes. You know their label since you construct the trees from the training set. You maximize the purity of the groups as much as possible each time you create a new node of the tree (meaning you cut your set in two). Of course, at the end of the tree, you want to have a clear answer.

The entroy at node $n$ is given by the following formula: $E(n) = - \sum_{k}p_{nk} log(p_{nk}) $, where $p_{nk}$ is the probability of class $k$.

A set is tidy if it contains only items with the same label, and messy if it is a mix of items with different labels. 

In [ ]:
p = np.random.random(100) # generates 100 random numbers between 0 and 1

In [ ]:
def entropy(p):
    
    return -p*np.log(p)-(1-p)*np.log(1-p)

In [ ]:
plt.scatter(p, entropy(p))

#### Information gain

Information gain (IG) measures how much information a feature gives us about the class. 
$$IG(feature) = <\text{entropy of parent node}> - <\text{weighted entropy of children}>$$

For a mathematical formula, check the related wikipedia page: https://en.wikipedia.org/wiki/Information_gain_in_decision_trees

In [ ]:
display(graphviz.Source(dot_graph))

In [ ]:
pe = entropy(159/426) # entropy of parent root
le = entropy(23/286) # entropy of left child node
re = entropy(136/140) # entropy of right child node
print(pe, le, re)

In [ ]:
weight_le = 286/426 # weight of left child node
weight_re = 140/426 # weight of right childe node
print(weight_le, weight_re)

In [ ]:
pe - (weight_le*le + weight_le*re) # GI

### Split strategies

The gini index and IG are used by Decision Tree Algorithm to construct a Decision Tree. ``DecisionTreeClassifier`` algorithm will always tries to maximize IG or minimize gini. A feature with highest IG will be tested first for splitting the data.

In [ ]:
tree = DecisionTreeClassifier(criterion="gini", max_depth=4, random_state=12)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

train_score = tree.score(X_train,y_train)
test_score = tree.score(X_test,y_test)
print("Score of train set: {}".format(train_score))
print("Score of test set: {}".format(test_score))

In [ ]:
tree = DecisionTreeClassifier(criterion="entropy", max_depth=4, random_state=12)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

train_score = tree.score(X_train,y_train)
test_score = tree.score(X_test,y_test)
print("Score of train set: {}".format(train_score))
print("Score of test set: {}".format(test_score))

### Hyperparameters

We have already used ``max_depth`` hyperparameter to prune the decision tree. There are others...
- ``min_samples_split`` is the minimum number of samples at any node. 
- ``min_samples_leaf`` is The minimum number of samples required to be at a leaf node. 
- ``max_features`` is the number of features to consider when looking for the best split.

In [ ]:
def display_min_samples_split(X_train, y_train, X_test, y_test, min_samples_split):
    train_score = []
    test_score = []
    for i in min_samples_split:
        tree = DecisionTreeClassifier(min_samples_split=i, criterion="entropy", random_state=12)
        tree.fit(X_train, y_train)
        y_pred = tree.predict(X_test)
        train_score.append(tree.score(X_train,y_train))
        test_score.append(tree.score(X_test,y_test))
    
    diff = [abs(x1 - x2) for (x1, x2) in zip(train_score, test_score)]    
    index = diff.index(min(diff))
    plt.plot(min_samples_split, train_score, label="training set")
    plt.plot(min_samples_split, test_score, label="test set") 
    plt.xlabel("min_samples_split")
    plt.ylabel("score")
    plt.legend()
    
    return min_samples_split[index], train_score[index], test_score[index]

In [ ]:
numbers = np.arange(2,100)

In [ ]:
display_min_samples_split(X_train, y_train, X_test, y_test, numbers)

In [ ]:
def display_min_samples_leaf(X_train, y_train, X_test, y_test, min_samples_leaf):
    train_score = []
    test_score = []
    for i in min_samples_leaf:
        tree = DecisionTreeClassifier(min_samples_leaf=i, criterion="entropy", random_state=12)
        tree.fit(X_train, y_train)
        y_pred = tree.predict(X_test)
        train_score.append(tree.score(X_train,y_train))
        test_score.append(tree.score(X_test,y_test))
    
    diff = [abs(x1 - x2) for (x1, x2) in zip(train_score, test_score)]    
    index = diff.index(min(diff))
    plt.plot(min_samples_leaf, train_score, label="training set")
    plt.plot(min_samples_leaf, test_score, label="test set") 
    plt.xlabel("min_samples_leaf")
    plt.ylabel("score")
    plt.legend()
    
    return min_samples_leaf[index], train_score[index], test_score[index]

In [ ]:
display_min_samples_leaf(X_train, y_train, X_test, y_test, numbers)

In [ ]:
len(cancer.feature_names)

In [ ]:
numbers = np.arange(1,len(cancer.feature_names)-1)

In [ ]:
def display_max_features(X_train, y_train, X_test, y_test, max_features):
    train_score = []
    test_score = []
    for i in max_features:
        tree = DecisionTreeClassifier(min_samples_leaf=i, criterion="entropy", random_state=12)
        tree.fit(X_train, y_train)
        y_pred = tree.predict(X_test)
        train_score.append(tree.score(X_train,y_train))
        test_score.append(tree.score(X_test,y_test))
        
    diff = [abs(x1 - x2) for (x1, x2) in zip(train_score, test_score)]    
    index = diff.index(min(diff))
    plt.plot(max_features, train_score, label="training set")
    plt.plot(max_features, test_score, label="test set") 
    plt.xlabel("max_features")
    plt.ylabel("score")
    plt.legend()
    
    return max_features[index], train_score[index], test_score[index]

In [ ]:
display_max_features(X_train, y_train, X_test, y_test,numbers)

### Feature importance

``DecisionTreeClassifier`` has an attribute ``feature_importances`` which rates how important each feature is for making decision. ``feature_importances`` is equal 0 for a feature $f$, it means that $f$ is not used at all. If close to 1, then $f$ does a good job at predicting the target. 

In [ ]:
tree = DecisionTreeClassifier(max_depth=3, random_state=0)
tree.fit(X_train, y_train)
tree.feature_importances_

In [ ]:
def plot_feature_importances_cancer(model):
    n_features = cancer.data.shape[1]
    plt.barh(np.arange(n_features), model.feature_importances_, align='center')
    plt.yticks(np.arange(n_features), cancer.feature_names)
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")
    plt.ylim(-1, n_features)

In [ ]:
plot_feature_importances_cancer(tree)

### Discussion 

- Advantages of decision tree: can be visualized, the logic is easy to understand, works with mixed dataset (continuous and discrete features), no need to standarize the data (e.g. one feature in cm and another one in inch).

- Disadvantages: Overfitting and poor generalization.

### Regression tree

Decision trees are used for regression, however, they cannot **extrapolate**, or make predictions outside the range of the target values.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston

In [ ]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns = boston.feature_names)
y = pd.DataFrame(boston.target, columns = ['Price'])

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=3)

In [ ]:
reg_tree = DecisionTreeRegressor(max_depth=3)
reg_tree.fit(X_train, y_train)
train_score = reg_tree.score(X_train,y_train)
test_score = reg_tree.score(X_test,y_test)
print("Score of train set: {}".format(train_score))
print("Score of test set: {}".format(test_score))

In [ ]:
LR = LinearRegression()
LR.fit(X_train, y_train)
train_score = LR.score(X_train,y_train)
test_score = LR.score(X_test,y_test)
print("Score of train set: {}".format(train_score))
print("Score of test set: {}".format(test_score))

In [ ]:
[(y_train.min()[0] <= pred and pred <= y_train.max()[0]) for pred in reg_tree.predict(X_test)]

In [ ]:
[(y_train.min()[0] <= pred and pred <= y_train.max()[0]) for pred in LR.predict(X_test)]

**Question.** In case of regression tree, what would be the predicted values for data outside the range of the training set? 

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(reg_tree, X, y, cv=5)

In [ ]:
cross_val_score(LR, X, y, cv=5)

Regression tree uses splitting strategies similar to classification tree plus the mse metric as a cost function. The predicted value is the average of the values in a class.

In [ ]:
reg_tree.predict(X_test)

In [ ]:
np.unique(reg_tree.predict(X_test))

In [ ]:
np.unique(LR.predict(X_test))

### Random forest

A random forest is a collection of decision trees. The random forest is one approach to address the overfitting issue in decision tree. Intuitively, if we build many trees each of which overfit in a different way, we can reduce the amount of overfitting by averaging their results. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
cancer = load_breast_cancer()

X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = pd.DataFrame(cancer.target, columns=["Type"])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=412)

tree = DecisionTreeClassifier(max_depth=3, random_state=12)
tree.fit(X_train, y_train.values.ravel()) # if I use y_train, 
#I get the following warning "A column-vector y was passed when a 1d array was expected. 
#Please change the shape of y to (n_samples,), for example using ravel()."

train_score = tree.score(X_train,y_train)
test_score = tree.score(X_test,y_test)
print("Score of train set (decision tree): {}".format(train_score))
print("Score of test set (decision tree): {}".format(test_score))

forest = RandomForestClassifier(n_estimators=5, random_state=12)
forest.fit(X_train, y_train.values.ravel())

train_score = forest.score(X_train,y_train)
test_score = forest.score(X_test,y_test)
print("Score of train set (random forest): {}".format(train_score))
print("Score of test set (random forest): {}".format(test_score))

To obtain different decision trees, we need different train datasets. ``RandomForestClassifier`` performs bootstraping, which create several datasets with some points are missing and other are repeated. Next, decision trees are built based on these new datasets. However, the algorithm is **random** meaning that instead of selecting the best feature at each node using gini or entropy, the algorithm randomly generate a subset of the features and selects the best feature from the subset. Hyperparameters  ``n_estimators`` and ``max_features`` specify the number of trees and the max number of randomly selected features.

In [ ]:
def display_n_estimators(X_train, y_train, X_test, y_test, n_estimators):
    train_score = []
    test_score = []
    for i in n_estimators:
        forest = RandomForestClassifier(n_estimators=i, random_state=12)
        forest.fit(X_train, y_train.values.ravel())
        train_score.append(forest.score(X_train,y_train))
        test_score.append(forest.score(X_test,y_test))
    
    diff = [abs(x1 - x2) for (x1, x2) in zip(train_score, test_score)]    
    index = diff.index(min(diff))
    plt.plot(n_estimators, train_score, label="training set")
    plt.plot(n_estimators, test_score, label="test set") 
    plt.xlabel("n_estimators")
    plt.ylabel("score")
    plt.legend()
    
    return n_estimators[index], train_score[index], test_score[index]

In [ ]:
numbers = np.arange(1,20)
display_n_estimators(X_train, y_train, X_test, y_test, numbers)

In [ ]:
def display_max_features(X_train, y_train, X_test, y_test, max_features):
    train_score = []
    test_score = []
    for i in max_features:
        forest = RandomForestClassifier(max_features=i, random_state=12)
        forest.fit(X_train, y_train.values.ravel())
        train_score.append(forest.score(X_train,y_train))
        test_score.append(forest.score(X_test,y_test))
    
    diff = [abs(x1 - x2) for (x1, x2) in zip(train_score, test_score)]    
    index = diff.index(min(diff))
    plt.plot(max_features, train_score, label="training set")
    plt.plot(max_features, test_score, label="test set") 
    plt.xlabel("n_estimators")
    plt.ylabel("score")
    plt.legend()
    
    return max_features[index], train_score[index], test_score[index]

In [ ]:
numbers = np.arange(1,20)
display_max_features(X_train, y_train, X_test, y_test, numbers)

In [ ]:
forest = RandomForestClassifier(n_estimators=5, random_state=12)
forest.fit(X_train, y_train.values.ravel())
plot_feature_importances_cancer(forest)

#### Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns = boston.feature_names)
y = pd.DataFrame(boston.target, columns = ['Price'])

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=3)

reg_forest = RandomForestRegressor(n_estimators=5,max_features=len(boston.feature_names))
reg_forest.fit(X_train, y_train.values.ravel())
train_score = reg_forest.score(X_train,y_train)
test_score = reg_forest.score(X_test,y_test)
print("Score of train set: {}".format(train_score))
print("Score of test set: {}".format(test_score))

#### Discussion

Random forests remedy the overfitting issue of decision tree. Furthermore, they share the same advantages as the decision tree (heteregenous data, no preprocessing, etc.), except, they are not easy to visualize and interpret. Time consuming random forests may be parallerized. By specifying ``n_jobs``, we can tell ``RandomForestClassifier`` or ``RandomForestRegressor`` how many CPU cores should be used. 

In [ ]:
reg_forest = RandomForestRegressor(n_estimators=3,n_jobs=2)
reg_forest.fit(X_train, y_train.values.ravel())

### Gradient boosted trees

Gradient boosted trees build decision trees in a serial manner, where each tree tries to correct the mistakes of the previous one. The learning rate controls how strongly each tree tries to correct the previous one. Gradient boosted trees use shallow decision trees (depth from 1 to 5). So, each decision tree provides a good prediction on a part of the data and more trees are added to improve the performance.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
cancer = load_breast_cancer()

X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y = pd.DataFrame(cancer.target, columns=["Type"])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=412)

boosted = GradientBoostingClassifier(random_state=0) # default n_estimator is 100
boosted.fit(X_train, y_train.values.ravel())

train_score = boosted.score(X_train,y_train)
test_score = boosted.score(X_test,y_test)
print("Score of train set (gradient boosted trees): {}".format(train_score))
print("Score of test set (gradient boosted trees): {}".format(test_score))

We are likely to be overfitting. To recover, we either lower the ``max_depth`` or lower the ``learning_rate``. According the sklearn documentation, the default values of ``max_depth`` and ``learning_rate`` are 3 and 0.1, respectively.

In [ ]:
boosted = GradientBoostingClassifier(max_depth=1, random_state=0)
boosted.fit(X_train, y_train.values.ravel())

train_score = boosted.score(X_train,y_train)
test_score = boosted.score(X_test,y_test)
print("Score of train set (gradient boosted trees): {}".format(train_score))
print("Score of test set (gradient boosted trees): {}".format(test_score))

In [ ]:
boosted = GradientBoostingClassifier(learning_rate=0.01, random_state=0)
boosted.fit(X_train, y_train.values.ravel())

train_score = boosted.score(X_train,y_train)
test_score = boosted.score(X_test,y_test)
print("Score of train set (gradient boosted trees): {}".format(train_score))
print("Score of test set (gradient boosted trees): {}".format(test_score))

#### Regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
boston = load_boston()
X = pd.DataFrame(boston.data, columns = boston.feature_names)
y = pd.DataFrame(boston.target, columns = ['Price'])

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=3)

reg_boosting = GradientBoostingRegressor(n_estimators=20,random_state=0)
reg_boosting.fit(X_train, y_train.values.ravel())
train_score = reg_boosting.score(X_train,y_train)
test_score = reg_boosting.score(X_test,y_test)
print("Score of train set: {}".format(train_score))
print("Score of test set: {}".format(test_score))

The book "Introduction to machine learning with python" recommend to first select a good ``n_estimator``, then search for optimal ``learning_rate``. 

In [ ]:
def display_n_estimators(X_train, y_train, X_test, y_test, n_estimators):
    train_score = []
    test_score = []
    for i in n_estimators:
        reg_boosting = GradientBoostingRegressor(n_estimators=i, random_state=0)
        reg_boosting.fit(X_train, y_train.values.ravel())
        train_score.append(reg_boosting.score(X_train,y_train))
        test_score.append(reg_boosting.score(X_test,y_test))
    
    diff = [(abs(x1 - x2),x1,x2,train_score.index(x1)+1) for (x1, x2) in zip(train_score, test_score) if x1 > 0.9 and x2 > 0.8]  
    mi = diff[0]
    for d in diff:
        if d[0] < mi[0]:
            mi = d
            
    plt.plot(n_estimators, train_score, label="training set")
    plt.plot(n_estimators, test_score, label="test set") 
    plt.xlabel("n_estimators")
    plt.ylabel("score")
    plt.legend()
    
    return mi

In [ ]:
numbers = np.arange(1,100)
display_n_estimators(X_train, y_train, X_test, y_test, numbers)

In [ ]:
reg_boosting = GradientBoostingRegressor(n_estimators=20,learning_rate=0.3,random_state=0)
reg_boosting.fit(X_train, y_train.values.ravel())
train_score = reg_boosting.score(X_train,y_train)
test_score = reg_boosting.score(X_test,y_test)
print("Score of train set: {}".format(train_score))
print("Score of test set: {}".format(test_score))